In [16]:
import os
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SemanticChunker
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from FlagEmbedding import BGEM3FlagModel
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableMap

ImportError: cannot import name 'SemanticChunker' from 'langchain.text_splitter' (/Users/kumarpersonal/Downloads/Inquiry-Assistant/venv/lib/python3.9/site-packages/langchain/text_splitter.py)

In [2]:
env_path = "/Users/kumarpersonal/Downloads/Inquiry-Assistant/venv/.env"
load_dotenv(dotenv_path=env_path)

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'inquiry-assistant'
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
corpus_path = "/Users/kumarpersonal/Downloads/Inquiry-Assistant/Context/cleaned_text.txt"
with open(corpus_path, "r", encoding="utf-8") as f:
    cleaned_text = f.read()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " "]
)
documents = splitter.create_documents([cleaned_text])

for doc in documents:
    doc.metadata["chunk_text"] = doc.page_content

In [ ]:
class BGEEmbedding:
    def __init__(self):
        self.model = BGEM3FlagModel('BAAI/bge-base-en', use_fp16=False)

    def embed_documents(self, texts):
        texts = ["passage: " + t for t in texts]
        output = self.model.encode(texts)
        dense_vecs = output["dense_vecs"]
        dense_vecs = dense_vecs / np.linalg.norm(dense_vecs, axis=1, keepdims=True)
        return dense_vecs.tolist() 

    def embed_query(self, text):
        output = self.model.encode(["query: " + text])
        dense_vec = output["dense_vecs"][0]
        dense_vec = dense_vec / np.linalg.norm(dense_vec)
        return dense_vec.tolist()

embed_model = BGEEmbedding()

In [ ]:
hf_encoder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5",
    encode_kwargs={"normalize_embeddings": True}
)

# Step 2: Initialize the semantic chunker
# chunk_size = number of tokens per chunk (semantic-based)


In [6]:
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(pinecone_index_name)

In [7]:
vectorstore = PineconeVectorStore(
    index=index,
    embedding=hf_encoder
)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5, 'fetch_k': 50, 'lambda_mult': 0.5})

In [8]:
llm = ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")

In [9]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an intelligent assistant for Scaler Academy, trained on internal documents, placement records, program curricula, and student feedback. "
     "You should answer questions only based on the context provided. "
     "If the answer is not found in the context, reply with: "
     "“I'm sorry, I couldn't find that information in the available documents.” "
     "Be precise, concise, and maintain a professional and helpful tone."
    ),
    ("human", 
     "Context:\n{context}\n\nQuestion:\n{question}")
])

output_parser = StrOutputParser()

In [10]:
retriever_runnable = RunnableLambda(lambda x: retriever.invoke(x["question"]))

In [11]:
def format_inputs(inputs):
    return {
        "context": "\n\n".join([doc.page_content for doc in inputs["documents"]]),
        "question": inputs["question"]
    }

In [12]:
rag_chain = (
    RunnableMap({
        "documents": retriever_runnable,
        "question": lambda x: x["question"]
    })
    | format_inputs | chat_prompt | llm | output_parser
)

In [15]:
query_text = input("Enter your question: ")
print("\nQuestion:", query_text)

response = rag_chain.invoke({"question": query_text})
print("\nAnswer:", response)


Question: What is Scaler?

Answer: Scaler Academy is a platform that offers tech-focused career programs. 

